<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

# Variáveis de input

+ **m** $\rightarrow$ Tamanho que vai ser atribuído aos BitVectors
+ **a** $\rightarrow$ Valor que vai ser atribuído ao x do estado inicial
+ **b** $\rightarrow$ Valor que vai ser atribuído ao y do estado inicial
+ **N** $\rightarrow$ Número de estados que vai gerar a partir do estado inicial
+ **M** $\rightarrow$ Número de estados que vai gerar a partir do estado de erro

# Variáveis auxiliares

+ **state** $\rightarrow$ Dicionário que guarda o estado e os valores de x, y, z


# Problema da multiplicação de inteiros positivos usando BitVectors

Neste exercício temos como objetivo implementar o algoritmo **"model-checking"** no problema da multiplicação de inteiros positivos usando BitVectors e, para isso, primeiramente, convertemos o grafo do exercício 1 do TP2 para código python, obtendo assim:

```Python
{ y = b, x = a, z = 0 }
0: while (y>0):
1:     if even(y):
2:         if (overflow(x*2)):
3:             raise Error          
4:         x = x*2;y = y/2
       elif odd(y):
5:         if(overflow(z+x)):
3:             raise Error
6:         y = y-1;z = z+x             
7: stop

```

# Implementação em python

In [8]:
from pysmt.shortcuts import *
from pysmt.typing import INT
from pysmt.typing import BVType
import itertools

m = 10        
a = 4
b = 3


## `genState`

Cria o dicionário **state**, que armazena o estado e os valores de **x**, **y** e **z**.

Recebe **vars**, **s** e **i** :
+ **vars** $\rightarrow$ Lista que contém o nome de cada elemento do state
+ **s** $\rightarrow$ String que vai ser associada a cada elemento do state
+ **i** $\rightarrow$ Inteiro que vai ser associada a cada elemento do state


In [2]:
def genState(vars,s,i):       
    state = {}
    state[vars[0]] = Symbol(vars[0]+'!'+s+str(i),BVType(m))
    state[vars[1]] = Symbol(vars[1]+'!'+s+str(i),BVType(m))
    state[vars[2]] = Symbol(vars[2]+'!'+s+str(i),BVType(m))
    state[vars[3]] = Symbol(vars[3]+'!'+s+str(i),BVType(m))
    return state

## `init`

Verifica se **state** é um estado inicial usando o seguinte predicado:

$$ (pc = 0 \wedge x = a \wedge y = b \wedge z = 0) $$

In [3]:
def init(state):
    return And(Equals(state['pc'],BV(0,m)),Equals(state['x'],BV(a,m)),
               Equals(state['y'],BV(b,m)),Equals(state['z'],BV(0,m)))

## `error`
Verifica se **state** é um estado de erro usando o seguitne predicado:

$$ (pc = 3) $$

In [4]:
def error(state):
    return Equals(state['pc'],BV(3,m))

## `trans`
Verifica que transformação é possível fazer entre os estados **curr** e **prox** usando o seguinte predicado:

$$ (pc = 0 \wedge pc' = 1 \wedge y > 0 \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 0 \wedge pc' = 7 \wedge y = 0 \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 1 \wedge pc' = 2 \wedge even(y) \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 2 \wedge pc' = 3 \wedge overflow(x*2) \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 2 \wedge pc' = 4 \wedge Not(overflow(x*2)) \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 4 \wedge pc' = 0 \wedge x' = x*2 \wedge y' = y/2 \wedge z' = z) $$
$$ \vee $$
$$ (pc = 1 \wedge pc' = 5 \wedge odd(y) \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 5 \wedge pc' = 3 \wedge overflow(x + z) \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 5 \wedge pc' = 6 \wedge Not(overflow(x + z)) \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 6 \wedge pc' = 0 \wedge y' = y - 1 \wedge z' = z + x \wedge x' = x) $$
$$ \vee $$
$$ (pc = 3 \wedge pc' = 7 \wedge x' = x \wedge y' = y \wedge z' = z) $$

In [5]:
def trans(curr,prox):
    t01 = And(Equals(curr['pc'],BV(0,m)), Equals(prox['pc'],BV(1,m)), 
              Equals(curr['x'],prox['x']),BVUGT(curr['y'],BV(0,m)),
              Equals(curr['y'],prox['y']),Equals(curr['z'],prox['z']))
    t07 = And(Equals(curr['pc'],BV(0,m)), Equals(prox['pc'],BV(7,m)), 
              Equals(curr['x'],prox['x']),Equals(curr['y'],BV(0,m)),
              Equals(curr['y'],prox['y']),Equals(curr['z'],prox['z']))
    t12 = And(Equals(curr['pc'],BV(1,m)), Equals(prox['pc'],BV(2,m)), 
              Equals(curr['x'],prox['x']),Equals(curr['y']&1,BV(0,m)&1),
              Equals(curr['y'],prox['y']),Equals(curr['z'],prox['z']))
    t23 = And(Equals(curr['pc'],BV(2,m)), Equals(prox['pc'],BV(3,m)), 
              Equals(curr['x']>>m-1 &1,BV(1,m)&1), Equals(prox['x'],curr['x']),
              Equals(prox['y'],curr['y']),Equals(curr['z'],prox['z']))
    t24 = And(Equals(curr['pc'],BV(2,m)), Equals(prox['pc'],BV(4,m)), 
              NotEquals(curr['x']>>m-1 &1,BV(1,m)&1), Equals(prox['x'],curr['x']),
              Equals(prox['y'],curr['y']),Equals(curr['z'],prox['z']))
    t40 = And(Equals(curr['pc'],BV(4,m)), Equals(prox['pc'],BV(0,m)), 
              Equals(prox['x'],curr['x']<<1), Equals(prox['y'],curr['y']>>1),
              Equals(curr['z'],prox['z']))
    t15 = And(Equals(curr['pc'],BV(1,m)), Equals(prox['pc'],BV(5,m)), 
              Equals(curr['x'],prox['x']),Equals(curr['y']&1,BV(1,m)&1),
              Equals(curr['y'],prox['y']),Equals(curr['z'],prox['z']))
    t53 = And(Equals(curr['pc'],BV(5,m)), Equals(prox['pc'],BV(3,m)), 
              Equals(curr['x'],prox['x']),Equals(prox['y'],curr['y']),
              Or(BVULT(BVSub(curr['x'],BV(2^m,m)),curr['z']),
              BVULT(BVSub(curr['z'],BV(2^m,m)),curr['x'])),
              Equals(curr['z'],prox['z']))
    t56 = And(Equals(curr['pc'],BV(5,m)), Equals(prox['pc'],BV(6,m)), 
              Equals(curr['x'],prox['x']),Equals(prox['y'],curr['y']),
              Equals(curr['z'],prox['z']))
    t60 = And(Equals(curr['pc'],BV(6,m)), Equals(prox['pc'],BV(0,m)), 
              Equals(curr['x'],prox['x']),Equals(prox['y'],BVSub(curr['y'],BVOne(m))),
              Equals(prox['z'],BVAdd(curr['z'],curr['x'])))
    t37 = And(Equals(curr['pc'],BV(3,m)), Equals(prox['pc'],BV(7,m)), 
              Equals(curr['x'],prox['x']),Equals(curr['y'],prox['y']),
              Equals(curr['z'],prox['z']))
    
    return Or(t01,t07,t12,t23,t24,t53,t56,t40,t15,t60,t37)

## `invert`

Inverte as transições.


## `same`

Verifica se 2 estados são iguais.

## `baseName`

Pega em todos os caracteres antes de um "!" num string.


## `rename`

Substitui o nome das varáveis de um certo interpolante pelas presentes no estado dado.

In [6]:
def invert(trans):
    return (lambda u, v : trans(v,u))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

## `model_checking`

Implementação do algoritmo *model checking*.

Recebe **vars**, **init**, **trans**, **error**, **N**, **M**,

+ **vars** $\rightarrow$ Lista com os nomes das varáveis existentes em cada estado ['pc','x','y','z']
+ **N** e **M** $\rightarrow$ Número de iterações que serão feitas pelo model checking a partir do estado inicial (*N*) e a partir do estado de erro (*M*)

Primeiro implementamos o *model checking* de forma usual onde, dando-lhe o número de iterações que deverá fazer a partir do estado inicial e do estado final, **N** e **M** respetivamente, devolve o que obtém nessas **N** $\times$ **M** iterações ou até encontrar uma situação onde o estado erro e o estado inicial se intersetam. Ou seja, nesta resolução *n* e *m* são incrementados automaticamente.

In [9]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name='z3') as s:
        
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1])
        
        for (n,m) in order:
             
            I = init(X[0])
            Tn = And([trans(X[i],X[i+1]) for i in range(n)])
            Rn = And(I,Tn)
            E = error(Y[0])
            Bm = And([invert(trans)(Y[i],Y[i+1]) for i in range(m)])
            Um = And(E,Bm)
            Vnm = And(Rn,same(X[n],Y[m]),Um)
            
            if s.solve([Vnm]):
                print(f"unsafe -> n:{n}, m:{m}")
            
            C = binary_interpolant(And(Rn,same(X[n],Y[m])), Um)
            
            if C is None:
                print("interpolant None")
                break
            
            C0 = rename(C,X[0])
            C1 = rename(C,X[1])
            T = trans(X[0],X[1])
            
            if not s.solve([C0,T,Not(C1)]):
                print(f"safe -> n:{n}, m:{m}")
                return 
            else:
                
                S = rename(C,X[n])
                while True:
                    A = And(S,trans(X[n],Y[m]))
                    if s.solve([A,Um]):
                        print("Não foi possível encontrar o majorante")
                        break
                    else:
                        Cnew = binary_interpolant(A,Um)
                        Cn = rename(Cnew,X[n])
                        
                        if s.solve([Cn,Not(S)]):
                            S = Or(S,Cn)
                        else:
                            print(f"safe -> n:{n}, m:{m}")
                            return                                
                    
        

model_checking(['pc','x','y','z'], init, trans, error,50, 50)

Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontrar o majorante
Não foi possível encontra

A seguir modificámos o código anterior de modo a caso não seja encontrado o majorante, em vez de *n* e *m* serem incrementados automaticamente, aparece um menu de texto que deixa o utilizador escolher qual das varáveis incrementar ou, se assim desejar, acabar a execução do programa.

In [10]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name='z3') as s:
        
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        n = 1
        m = 1
        while n <= N and m <= M:
             
            I = init(X[0])
            Tn = And([trans(X[i],X[i+1]) for i in range(n)])
            Rn = And(I,Tn)
            E = error(Y[0])
            Bm = And([invert(trans)(Y[i],Y[i+1]) for i in range(m)])
            Um = And(E,Bm)
            Vnm = And(Rn,same(X[n],Y[m]),Um)
            
            if s.solve([Vnm]):
                print(f"unsafe -> n:{n}, m:{m}")
            
            C = binary_interpolant(And(Rn,same(X[n],Y[m])), Um)
            
            if C is None:
                print("interpolant None")
                break
            
            C0 = rename(C,X[0])
            C1 = rename(C,X[1])
            T = trans(X[0],X[1])
            
            if not s.solve([C0,T,Not(C1)]):
                print(f"safe -> n:{n}, m:{m}")
                return 
            else:
                
                S = rename(C,X[n])
                while True:
                    A = And(S,trans(X[n],Y[m]))
                    if s.solve([A,Um]):
                        next = input("Não foi possível encontrar o majorante\nEscolha a próxima ação:\n1 - Incrementar n\n2 - Incrementar m\n3 - Terminar a execução\n")
                        if next == "1":
                            n+=1
                        elif next == "2":
                            m+=1
                        else:
                            return
                        break
                    else:
                        Cnew = binary_interpolant(A,Um)
                        Cn = rename(Cnew,X[n])
                        
                        if s.solve([Cn,Not(S)]):
                            S = Or(S,Cn)
                        else:
                            print(f"safe -> n:{n}, m:{m}")
                            return                                
                    
        

model_checking(['pc','x','y','z'], init, trans, error,50, 50)  

Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2
Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
2


## Testes

In [ ]:
-----------------------------|

a = 7
b = 2
m = 3

------

Não foi possível encontrar o majorante
Escolha a próxima ação:
1 - Incrementar n
2 - Incrementar m
3 - Terminar a execução
1
unsafe -> n:2, m:1
interpolant None

-----------------------------|

m = 10        
a = 4
b = 5

------

Não foi possível encontrar o majorante    |
Escolha a próxima ação:                   |
1 - Incrementar n                         |
2 - Incrementar m                         | repetiu 21 vezes
3 - Terminar a execução                   |  
1                                         |
safe -> n:22, m:1

-----------------------------|

m = 10        
a = 4
b = 6

------

Não foi possível encontrar o majorante    |
Escolha a próxima ação:                   |
1 - Incrementar n                         |
2 - Incrementar m                         | repetiu 13 vezes
3 - Terminar a execução                   |
2                                         |
unsafe -> n:1, m:14                       
interpolant None

-----------------------------|

m = 10        
a = 4
b = 6

------

Não foi possível encontrar o majorante    |
Escolha a próxima ação:                   |
1 - Incrementar n                         |
2 - Incrementar m                         | repetiu 13 vezes
3 - Terminar a execução                   |
1                                         |
unsafe -> n:14, m:1                       
interpolant None

-----------------------------|

m = 10        
a = 8
b = 0

------

safe -> n:1, m:1

-----------------------------|